In [1]:
## FOR GENERAL UNIX FUNCTIONS SEE trace-unix-general.ipynb 
## STARTS FROM LINE 1687 (tomohara-aliases.bash)

## Bracketed Paste is disabled to prevent characters after output
## Example: 
# $ echo 'Hi'
# | Hi?2004l

bind 'set enable-bracketed-paste off'

In [2]:
## TEST:  Make sure simple prompt used (e.g., no escapes that might contaminate output)
## PS1="$ "
## TODO: PS1="> "

## NOTE: The Jupyter bash kernel requires that PS1 not be modified as they customize it. 
echo $PS1

[PEXP\[\]ECT_PROMPT>


In [3]:
# Delete all aliases and function
# TODO: Instead start from pristine environment
unalias -a
alias | wc -l
for f in $(typeset -f | egrep '^\w+'); do unset -f $f; done
typeset -f | egrep '^\w+' | wc -l

0
0


In [4]:
# Setting a temp directory for tests
TMP=/tmp/test-unix-alias

## NOTE: Source it directly from the ./tests directory.
BIN_DIR=$PWD/..

alias | wc -l

0


In [5]:
## NOTE: For reproducability, the directory name needs to be fixed
## In place of $$, use a psuedo random number (e,g., 2899)
## *** All output from one run to the next needs to be the same ***

## temp_dir=$TMP/test-$$
temp_dir=$TMP/test-2899

mkdir -p "$temp_dir"
# TODO: /bin/rm -rvf "$temp_dir"
cd "$temp_dir"
pwd

#ALIAS FOR PRINTING SEPERATION LINES (FOR JUPYTER)
alias linebr="printf '%*s\n' "${COLUMNS:-$(tput cols)}" '' | tr ' ' -"

/tmp/test-unix-alias/test-2899


In [6]:
# Count aliases proper
alias | wc -l
# Count functions
typeset -f | egrep '^\w+' | wc -l

1
0


In [7]:
function group-members () { ypcat group | $GREP -i "$1"; }

In [8]:
## ERROR I - group-members

# $ group-members
# | bash: ypcat: command not found
# | bash: -i: command not found

In [9]:
## TEST 1 - do-make
function do-make () { /bin/mv -f _make.log _old_make.log; make "$@" >| _make.log 2>&1; $PAGER _make.log; }

In [10]:
## 1A) do-make
rm -rf ./*

echo "Hi Mom!" > himom.txt
echo "Hi Dad!" > hidad.txt

linebr
do-make "himom.txt"
## THIS SHOWS THE LATEST LOG OF do-make
linebr

cat _make.log
do-make "hidad.txt"
cat _make.log

linebr

ls -l

--------------------------------------------------------------------------------
/bin/mv: cannot stat '_make.log': No such file or directory
make: Nothing to be done for 'himom.txt'.
--------------------------------------------------------------------------------
make: Nothing to be done for 'himom.txt'.
make: Nothing to be done for 'hidad.txt'.
make: Nothing to be done for 'hidad.txt'.
--------------------------------------------------------------------------------
total 16
-rw-rw-r-- 1 aveey aveey  8 Sep  7 23:19 hidad.txt
-rw-rw-r-- 1 aveey aveey  8 Sep  7 23:19 himom.txt
-rw-rw-r-- 1 aveey aveey 42 Sep  7 23:19 _make.log
-rw-rw-r-- 1 aveey aveey 42 Sep  7 23:19 _old_make.log


In [11]:
## TEST 2 - MERGING ALIASAS & FUNCTIONS

# $ man merge
#   merge [ options ] file1 file2 file3
#   merge  incorporates all changes that lead from file2 to file3 into file1.
# NOTE: merge -p mod-file1 original mod-file2 >| new-file
alias merge='echo "do-merge MODFILE1 OLDFILE MODFILE2 > NEWFILE"'
alias do-merge='/usr/bin/merge -p'
# note: version of merge usinf diff3 to specify diff program (shell wrapper with whitespace ignored)
alias diff3-merge='/usr/bin/diff3 --merge --text --diff-program=diff.sh'
## TODO: --auto
function kdiff-merge() {
    if [ "$3" = "" ]; then
        echo "usage: $0 changed1 old changed2 output"
        return
    fi
    kdiff3 --merge --output "$4" "$1" "$2" "$3"
}

In [12]:
# 2A) merge REPLACED BY do-merge 

## INITIAL FILES TO TEST THE ALIASES
touch abc def ghi

# MESSAGE SHOWN IF MERGE IS USED
merge abc def ghi

do-merge MODFILE1 OLDFILE MODFILE2 > NEWFILE abc def ghi


In [13]:
# 2B) do-merge

## ERROR II - do-merge DOESN'T SEEM TO WORK
# $ do-merge abc def ghi
# | bash: /usr/bin/merge: No such file or directory

In [14]:
# 2C) diff3-merge

## ERROR III - Undefined variable (TMP)

# $ diff3-merge abc def ghi
# | TMP: Undefined variable.
# | TMP: Undefined variable.

In [15]:
# 2D) kdiff-merge

#kdiff-merge "abc" "def" "ghi" "himom.txt"
## ERROR IV - kdiff-merge NOT WORKING AS kdiff3 COMMAND MISSING
# $ kdiff-merge abc def ghi
# | bash: kdiff3: command not found

In [16]:
# TEST 3 - WHICH FUNCTION
function which { /usr/bin/which "$@" 2> /dev/null; }
#
# full-dirname(filename): returns full path of directory for file
#
function full-dirname () { local dir=$(dirname $1); case $dir in .*) dir="$PWD/$1";; esac; echo $dir; }
#
# base-name-with-dir(file, suffix): version of basename include dir
function basename-with-dir {
    local file="$1"
    local suffix="$2"
    echo $(dirname "$file")/$(basename "$file" "$suffix");
}

In [17]:
# 3A) full-dirname RETURNS THE FULL PATH OF THE FILE
ls
linebr
full-dirname "himom.txt"

abc  def  ghi  hidad.txt  himom.txt  _make.log	_old_make.log
--------------------------------------------------------------------------------
/tmp/test-unix-alias/test-2899/himom.txt


In [18]:
# 3B) base-name-with-dir RETURNS THE BASENAME INCLUDING dir
basename-with-dir "himom.txt" 2

./himom.txt


In [19]:
## 3C) rpm-extract IS USED FOR RED HAT BASED DISTROS (RHEL, Fedora) 
function rpm-extract () { rpm2cpio "$1" | cpio --extract --make-directories; }

# # ERROR V:
# $ rpm-extract
# | bash: rpm2cpio: command not found
# | cpio: premature end of archive


In [20]:
# TEST 4 - DEVICE INFO ALIASES
alias dump-url='wget --recursive --relative'
#
alias gtime='/usr/bin/time'

# see 'man 5 os-release'
alias linux-version="cat /etc/os-release"
alias os-release=linux-version
alias system-status='system_status.sh -'
#

In [21]:
## 4A) dump-url GIVES 
dump-url https://www.google.com/

--2022-09-07 23:19:22--  https://www.google.com/
Resolving www.google.com (www.google.com)... 142.250.195.132, 2404:6800:4007:825::2004
Connecting to www.google.com (www.google.com)|142.250.195.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘www.google.com/index.html’

www.google.com/inde     [ <=>                ]  16.11K  --.-KB/s    in 0.009s  

2022-09-07 23:19:22 (1.68 MB/s) - ‘www.google.com/index.html’ saved [16499]

FINISHED --2022-09-07 23:19:22--
Total wall clock time: 0.4s
Downloaded: 1 files, 16K in 0.009s (1.68 MB/s)


In [22]:
## 4B) gtime
# ERROR - VI 

# $ gtime
# | /usr/bin/time: missing program to run
# | Try '/usr/bin/time --help' for more information


In [23]:
## 4C) linux-version GIVES INFO ALL INFO ABOUT THE OS INSTALLED
linux-version
linebr
# os-release ALSO WORKS SAME AS linux-version
os-release

PRETTY_NAME="Ubuntu 22.04.1 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.1 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy
--------------------------------------------------------------------------------
PRETTY_NAME="Ubuntu 22.04.1 LTS"
NAME="Ubuntu"
VERSION_ID="22.04"
VERSION="22.04.1 LTS (Jammy Jellyfish)"
VERSION_CODENAME=jammy
ID=ubuntu
ID_LIKE=debian
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
UBUNTU_CODENAME=jammy


In [24]:
## 4D) system-status

# $ system-status (# STUCK DUE TO OSTYPE: Undefined variable.)

In [25]:
## 5A)
# propos-command: show apropos output for lExecutable programs or shell commands (i.e., man section 1)
# TODO: use '$@' for '$*' (or note why not appropriate)
# TODO: 'apropos --section 1'
# EX: apropos-command time | grep asctime | wc -l => 0
function apropos-command () { apropos $* 2>&1 | $GREP '(1)' | $PAGER; }
#
# 
function split-tokens () { perl -pe "s/\s+/\n/g;" "$@"; }
alias tokenize='split-tokens'
#

In [26]:
# tokenize SPLITS THE CONTENTS IN SEVERAL LINES
tokenize _old_make.log

make:
Nothing
to
be
done
for
'himom.txt'.


In [27]:
# 5B)
# perl-echo(arg1): print arg1 via perl
# NOTE: Usually Bash $'string' special tokens can be used instead
#       (e.g., perl-echo "A\tB" => echo $'A\tB').
# TODO: try to minimize use of quotes in perl-echo (e.g., need to mix single with double) due to bash quirks
function perl-echo () { perl -e 'print "'$1'\n";'; }
## MISC
function perl-echo-sans-newline () { perl -e 'print "'$1'";'; }
#

In [28]:
# perl-echo ONLY PRINTS OUT THE 1ST ARGUMENT
PERL_PRINT="This is Ubuntu!"
perl-echo $PERL_PRINT
perl-echo-sans-newline $PERL_PRINT

This
This

In [29]:
# 5C)
# perl-printf(format, arg): output ARG2 via perl format ARG1
## TODO: generalize perl-printf to more than 2 args
## function perl-printf () { perl -e 'printf "$1\n", @_[1..$#_];';  }
##
## function perl-printf () { perl -e "printf \"$1\"", qw/$2 "$3" "$4" "$5" "$6" "$7" "$8" "$9"/; }
## MISC
function perl-printf () { perl -e "printf \"$1\", $2;"; }
##
## TODO: get following to work for 'perl-print "how now\nbrown cow\n"'
## function perl-print () { perl -e "print $1"; -e 'print "\n";'; }
## MISC
function perl-print () { perl -e "printf \"$1\";" -e 'print "\n";'; }
## MISC
function perl-print-n () { perl -e "printf \"$1\";"; }
#
# quote-tokens(): puts double quote around each token on commnd line
# note: used to circumvent google search's annoying search term dropping
function quote-tokens () { echo "$@" | perl -pe 's/(\S+)/"\1"/g;'; }

In [30]:
perl-printf "ONE KISS IS ALL IT TAKES\n"
linebr
perl-print "2\n3\n5\n7\n11"
linebr
perl-print-n "A B C D E F G\n"
linebr
quote-tokens "HELP ME!"

ONE KISS IS ALL IT TAKES
--------------------------------------------------------------------------------
2
3
5
7
11
--------------------------------------------------------------------------------
A B C D E F G
--------------------------------------------------------------------------------
"HELP" "ME!"


In [31]:
# 6A)
# Unix/Win32 networking aliases
if [ "$OSTYPE" != "cygwin" ]; then alias ipconfig=ifconfig; fi
alias set-display-local='export DISPLAY=localhost:0.0'

set-display-local
echo $DISPLAY

localhost:0.0


In [32]:
# 7A)
# Bash aliases
# Note: '- -o' is idiom for turning off
alias bash-trace-on='set -o xtrace'
alias bash-trace-off='set - -o xtrace'
#
# trace-cmd(command-line): runs command-line with bash tracing enable to
# show argument expansion with result piped into less
function trace-cmd() {
    (
        echo "start: $(date)";
        bash-trace-on; 
        eval "$@"; 
        bash-trace-off;
        echo "end: $(date)";
    ) 2>&1 | $PAGER;
}
## ALT: function trace-cmd() { bash-trace-on; @_; bash-trace-off; }
alias cmd-trace='trace-cmd'


In [33]:
# bash-trace-on ENABLES bash tracing
bash-trace-on
# trace-cmd TRACES COMMANDS AND STATS (TIME) OF THE USER
trace-cmd
# bash-trace-on DISABLES bash tracing
bash-trace-off
linebr
trace-cmd

+ trace-cmd
+ cat
++ date
+ echo 'start: Wed Sep  7 11:19:28 PM +0545 2022'
start: Wed Sep  7 11:19:28 PM +0545 2022
+ set -o xtrace
+ eval
+ set - -o xtrace
end: Wed Sep  7 11:19:28 PM +0545 2022
+ set - -o xtrace
--------------------------------------------------------------------------------
start: Wed Sep  7 11:19:28 PM +0545 2022
+ eval
+ set - -o xtrace
end: Wed Sep  7 11:19:28 PM +0545 2022


In [34]:
# cmd-usage(command): output usage for command to _command.list (with spaces
# replaced by underscores)
function cmd-usage () {
    local command="$@"
    local usage_file="_$(echo "$command" | tr ' ' '_')-usage.list"
    $command --help  2>&1 | ansifilter > "$usage_file"
    $PAGER_NOEXIT "$usage_file"
}

## ERROR DETECTED
# $ cmd-usage dir
# | bash: ansifilter: command not found
# | bash: _dir-usage.list: command not found

In [35]:
# 9A)
# Compressing/uncompressing a subdirectory tree (ignoring symbolic links) 
# TODO: write scripts for this (given the complexity)
# TODO: don't uncompress compressed archives (.tar.gz files)
function compress-dir() {
    log_file=$TEMP/compress_$(basename "$1").log;
    find "$1" \( -not -type l \) -exec gzip -vf {} \; >| "$log_file" 2>&1; 
    $PAGER "$log_file";
}
# NOTE: zipped archived are kept compressed
function uncompress-dir() {
    log_file=$TEMP/uncompress_$(basename "$1").log;
    find "$1" \( -not -type l \) \( -not -name \*.tar.gz \) -exec gunzip -vf {} \; >| "$log_file" 2>&1; $PAGER "$log_file";
}
alias compress-this-dir='compress-dir $PWD'
alias ununcompress-this-dir='uncompress-dir $PWD'


In [36]:
## ERROR (PERMISSION DENIED - NEED TO STUDY)
# compress-this-dir ./*
# linebr
# ls
# linebr
# ununcompress-this-dir ./*


bash: /compress_test-2899.log: Permission denied
cat: /compress_test-2899.log: No such file or directory
--------------------------------------------------------------------------------
abc  def  ghi  hidad.txt  himom.txt  _make.log	_old_make.log  www.google.com
--------------------------------------------------------------------------------
bash: /uncompress_test-2899.log: Permission denied
cat: /uncompress_test-2899.log: No such file or directory


: 1

In [ ]:
# TEST 9B)
alias old-count-exts='$LS | count-it "\.[^.]*\w" | sort $SORT_COL2 -rn | $PAGER'
function count-exts () { $LS | count-it '\.[^.]+$' | sort $SORT_COL2 -rn | $PAGER; }

alias kill-iceweasel='kill_em.sh iceweasel'

In [ ]:
# ERROR GENERATED: count-it NOT MENTIONED IN THIS TEST
# $ old-count-exts
# | bash: count-it: command not found

# $ kill-iceweasel
# | pattern=/:[0-9][0-9] [^ ]*iceweasel/
# | filter=/($)(^)/
# | OSTYPE: Undefined variable.

kill-iceweasel